# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [5]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [6]:
salaries = pd.read_csv(r'C:\Users\adria\Documents\GitHub\lab-hypothesis-testing-en\data\Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [7]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [8]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
from scipy.stats import ttest_1samp
#Sample 30 workers from chicago
sample_1 = salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate'].sample(30)
sample_1

#H0: \mu_1 = 30usd/h
#H1 \mu_1 != 30usd/h

#one-sample t-test
hypothesized_mean = 30 # as in 30usd hourrly
t_stat, p_value = ttest_1samp(sample_1, hypothesized_mean)

print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpretation at 95% confidence level
alpha = 0.05 # standard value in statistics unless we need broader like 0.10 or more stringent like 0.001
if p_value < alpha:
    print("Reject the null hypothesis: The hourly wage is significantly different from $30/hr.")
else:
    print("Fail to reject the null hypothesis: The hourly wage is not significantly different from $30/hr.")

t-statistic: 1.0883
p-value: 0.2854
Fail to reject the null hypothesis: The hourly wage is not significantly different from $30/hr.


In [10]:

#From chatgpt
"""
# Sample statistics
sample_mean = np.mean(sample_1)
sample_std = np.std(sample_1, ddof=1)  # Sample standard deviation
n = len(sample_1)
hypothesized_mean = 30

# Calculate the t-statistic
t_stat = (sample_mean - hypothesized_mean) / (sample_std / np.sqrt(n))

# Degrees of freedom so sample size minus 1
df = n - 1

# Calculate the p-value for the two-tailed test
p_value = 2 * t.sf(np.abs(t_stat), df)

# Print data
print(f"Sample mean: {sample_mean:.2f}")
print(f"Sample standard deviation: {sample_std:.2f}")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: The hourly wage is significantly different from $30/hr.")
else:
    print("Fail to reject the null hypothesis: The hourly wage is not significantly different from $30/hr.")
"""

'\n# Sample statistics\nsample_mean = np.mean(sample_1)\nsample_std = np.std(sample_1, ddof=1)  # Sample standard deviation\nn = len(sample_1)\nhypothesized_mean = 30\n\n# Calculate the t-statistic\nt_stat = (sample_mean - hypothesized_mean) / (sample_std / np.sqrt(n))\n\n# Degrees of freedom so sample size minus 1\ndf = n - 1\n\n# Calculate the p-value for the two-tailed test\np_value = 2 * t.sf(np.abs(t_stat), df)\n\n# Print data\nprint(f"Sample mean: {sample_mean:.2f}")\nprint(f"Sample standard deviation: {sample_std:.2f}")\nprint(f"t-statistic: {t_stat:.4f}")\nprint(f"p-value: {p_value:.4f}")\n\n# Interpretation\nalpha = 0.05\nif p_value < alpha:\n    print("Reject the null hypothesis: The hourly wage is significantly different from $30/hr.")\nelse:\n    print("Fail to reject the null hypothesis: The hourly wage is not significantly different from $30/hr.")\n'

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [11]:
#Sample 30 workers from chicago
sample_1 = salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate'].sample(30)
sample_1

# Sample statistics
sample_mean = np.mean(sample_1)
sample_std = np.std(sample_1, ddof=1)
n = len(sample_1)

# Calculate the standard error of the mean (SEM)
standard_error = sem(sample_1)

# Calculate the 95% confidence interval
confidence_level = 0.95
degrees_of_freedom = n - 1

confidence_interval = t.interval(confidence_level, degrees_of_freedom, loc=sample_mean, scale=standard_error)


print(f"Sample mean: {sample_mean}")
print(f"Sample standard error: {standard_error}")
print(f"95% Confidence interval: {confidence_interval}")

Sample mean: 31.451333333333334
Sample standard error: 1.828329476415425
95% Confidence interval: (27.711979692583544, 35.19068697408313)


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [16]:
from statsmodels.stats.proportion import proportions_ztest

#Null hypothesis he proportion of hourly workers in the City of Chicago is 25%

#Alternative hypothesis the proportion of hourly workers in the City of Chicago is not equal to 25%

#number of hourly workers and the totalworkers
n_h_workers = salaries[salaries['Department'] == 'Hourly']['Name'].count()
total_workers = salaries['Name'].count()

#null hypothesis proportion
null_proportion = 0.25

#hypothesis test
z_stat, p_value = proportions_ztest(n_h_workers, total_workers, value=null_proportion)

print("Z-statistic:", z_stat)
print("P-value:", p_value)

Z-statistic: -inf
P-value: 0.0


The z-statistic being negative infinity suggests that the observed proportion of hourly workers is significantly lower than the hypothesized proportion of 25%. 
the p-value being 0.0 indicates strong evidence against the null hypothesis.